In [2]:
import cv2
import os
# # Đọc tất cả ảnh trong file lưu trữ lại
# Đường dẫn tới thư mục chứa ảnh
def read_image(folder_path):
    # Khởi tạo một danh sách để lưu trữ các hình ảnh
    images = []
    
    # Lặp qua tất cả các tệp trong thư mục
    for filename in os.listdir(folder_path):
        # Xác định đường dẫn đầy đủ đến tệp
        file_path = os.path.join(folder_path, filename)
        # Đảm bảo rằng tệp là một tệp ảnh
        if os.path.isfile(file_path) and any(filename.endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
            # Đọc ảnh và thêm vào danh sách
            image = cv2.imread(file_path)
            if image is not None:
                images.append(image)
    return images

In [5]:
# # Hàm trích rút Color Histogram
import numpy as np
from skimage import feature
import cv2
import math
def my_calcHist(image, channels, histSize, ranges):
    # Khởi tạo histogram với tất cả giá trị bằng 0
    hist = np.zeros(histSize, dtype=np.int64)
    # Lặp qua tất cả các pixel trong ảnh
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            # Lấy giá trị của kênh màu được chỉ định
            bin_vals = [image[i, j, c] for c in channels]
            # Tính chỉ số của bin
            bin_idxs = [(bin_vals[c] - ranges[c][0]) * histSize[c] // (ranges[c][1] - ranges[c][0]) for c in range(len(channels))]
            # Tăng giá trị của bin tương ứng lên 1
            hist[tuple(bin_idxs)] += 1
    return hist
def euclidean_distance(vector1, vector2):
    # Chuyển đổi list sang numpy array để sử dụng tính năng của numpy
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)
    
    # Tính khoảng cách Euclidean
    distance = np.sqrt(np.sum((vector1 - vector2)**2))
    return distance
def convert_image_rgb_to_gray(img_rgb):
    # Get the height (h), width (w), and number of channels (_) of the input RGB image
    h, w, _ = img_rgb.shape

    # Create an empty numpy array of zeros with dimensions (h, w) to hold the converted grayscale values
    img_gray = np.zeros((h, w), dtype=np.uint32)

    # Convert each pixel from RGB to grayscale using the formula Y = 0.299R + 0.587G + 0.114B
    for i in range(h):
        for j in range(w):
            r, g, b = img_rgb[i, j]
            gray_value = int(0.299 * r + 0.587 * g + 0.114 * b)
            img_gray[i, j] = gray_value
            # Return the final grayscale image as a numpy array
    return np.array(img_gray)

def hog_feature(gray_img):  # default gray_image
    # Compute the HOG features and the HOG visualization image using the scikit-image "feature" module's hog() function.
    (hog_feats, hogImage) = feature.hog(
        gray_img,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        transform_sqrt=True,
        block_norm="L2",
        visualize=True,
    )

    # Return the HOG feature descriptor as a numpy array
    return hog_feats
def feature_extraction(img):  # RGB image
    bins = [16, 16, 16]
    ranges = [[0, 256], [0, 256], [0, 256]]
    hist_my = my_calcHist(img, [0, 1, 2], bins, ranges)
    embedding = np.array(hist_my.flatten()) / (256*256)

    gray_image = convert_image_rgb_to_gray(img)
    embedding_hog = list(hog_feature(gray_image))
    return embedding,embedding_hog
def image_distance(img1,img2):
    color_hist_img1 , hog_img1 = feature_extraction(img1)
    color_hist_img2 , hog_img2 = feature_extraction(img2)
    # khoảng cách sẽ năm trong khoảng từ 0 - căn 2
    color_distance = euclidean_distance(color_hist_img1,color_hist_img2)
    ##chuyển nó về khoảng từ 0-1 bằng cách chia cho căn 2
    color_distance = color_distance/ math.sqrt(2)
    ##Giá trị sẽ nằm trong khoảng từ 0-căn bậc 2 của n
    hog_distance = euclidean_distance(hog_img1,hog_img2)
    ## chuyển về khoảng từ 0-1
    hog_distance =  hog_distance / 186
    #Trả về trung bình khoảng cách
    return (color_distance + hog_distance)/2
def search_image(image):
    images = read_image('resize_image')
    distances = []
    # Duyệt qua từng ảnh trong mảng images
    for img in images:     
        # Tính khoảng cách giữa ảnh hiện tại và ảnh được truyền vào
        distance = image_distance(img, image )
        print(distance)
        # Thêm tuple chứa khoảng cách và ảnh hiện tại vào danh sách distances
        distances.append((distance, img))
        
    # Sắp xếp danh sách distances theo khoảng cách tăng dần
    distances.sort(key=lambda x: x[0])
    
    # Trả về 5 ảnh có khoảng cách ngắn nhất
    closest_images = [img for _, img in distances[:5]]
    return closest_images

In [12]:
import cv2
bgr_image1 = cv2.imread('test.jpg')
print("1")
image1 = cv2.cvtColor(bgr_image1, cv2.COLOR_BGR2RGB)
output = search_image(image1)
for i, img in enumerate(output):
    # Hiển thị ảnh từ mảng NumPy
    cv2.imshow(f"Image {i+1}", img)
    cv2.waitKey(0)  # Chờ cho đến khi nhấn một phím bất kỳ
    cv2.destroyAllWindows()

1
0.12784483360194943
0.123953867048993
0.12259802202925528
0.1312471392368698
0.14444615484216095
0.11093525821895567
0.1287295334999978
0.13715572834301898
0.11970951041300124
0.07022452805129364
0.11632420104234903
0.19662191133692125
0.11399369984911534
0.1536205735463448
0.12356354366661337
0.12119489021625293
0.11977453930000859
0.12774223722523476
0.11901954151177183
0.11352292609224157
0.13147924873206007
0.13420788205436704
0.13342363406991384
0.1527373169462823
0.1366058932318977
0.11353421419739983
0.12974172510442944
0.12132797320886528
0.12602075480402397
0.16006005027802
0.11107526985773432
0.15592454989839977
0.15026501089982136
0.11791002059010162
0.12705245606468452
0.1344257953215287
0.10612653872088126
0.13271690941403805
0.1195373698598498
0.13172295037576634
0.1325392752646106
0.11802746150222912
0.1323746240515956
0.12972664701739503
0.12000207788453068
0.11504820764692847
0.11683866160863651
0.12492792472513072
0.1436310773042832
0.13307941691104075
0.11941010378